# Решение

## Подготовка

Импортируем необходимые для начала работы библиотеки

In [ ]:
import os
import numpy as np
import pandas as pd

Определяем папку, где находятся данные

In [ ]:
data_dir = os.path.join('..', 'data')

Загружаем данные

In [ ]:
users_df = pd.read_csv(os.path.join(data_dir, 'users.tsv'), sep='\t')
history_df = pd.read_csv(os.path.join(data_dir, 'history.tsv'), sep='\t')
validate_df = pd.read_csv(os.path.join(data_dir, 'validate.tsv'), sep='\t')
validate_answers_df = pd.read_csv(os.path.join(data_dir, 'validate_answers.tsv'), sep='\t')

In [ ]:
from sklearn.preprocessing import FunctionTransformer

log_transformer = FunctionTransformer(np.log1p)

## Работа с "Историей"

Чтобы удобнее было перезапускать частично, периодически будем копировать датафреймы

In [ ]:
history = history_df.copy()

In [ ]:
history = history[history['cpm'] < history['cpm'].quantile(0.95)]
history = history[history['cpm'] > history['cpm'].quantile(0.05)]
history['cpm'] = log_transformer.transform(history['cpm'])

Проходимся по датафрейму и агрегируем необходимые нам данные

Можно заменить на вызовы методов датафреймов для необходимых агрегаций, но мне показалось, что так может быть быстрее

Основные показатели которые нам нужны - набор выигравших cpm для каждого пользователя и распределение пользователей по площадкам 

In [ ]:
user_cpms = {}
publisher_users = {}

for index, row in history.iterrows():
    user_id = str(int(row['user_id']))
    publisher = str(int(row['publisher']))
    cpm = row['cpm']

    if user_id not in user_cpms:
        user_cpms[user_id] = []
    user_cpms[user_id].append(cpm)

    if publisher not in publisher_users:
        publisher_users[publisher] = set()
    publisher_users[publisher].add(user_id)

## Работа с "Рекламой"

Аналогично копируем датафрейм

In [ ]:
ads = validate_df.copy()

In [ ]:
ads['cpm'] = log_transformer.transform(ads['cpm'])

Создаем фичи  
В принципе тут можно без комментариев понять по названиям

In [ ]:
ads['hours'] = ads.apply(lambda row: row['hour_end'] - row['hour_start'], axis=1)

ads['cpm_x_hours'] = ads['cpm'] * ads['hours']

Далее мы начинаем обходить для каждого рекламного объявления пользователей, которые в нем указаны и создаем фичи на их основе

Как показала практика это одни из самых значимых фичей

`users_power` - это показатель того, сколько человек смотрит рекламы  
не придумал более хорошего названия

In [ ]:
def process_ad_users(user_ids, user_cpms):
    users = user_ids.split(',')
    counts = []
    cpms = []
    for user in users:
        if user in user_cpms:
            counts.append(len(user_cpms[user]))
            cpms.append(np.mean(user_cpms[user]))

    return sum(counts), np.mean(counts), np.mean(cpms)

In [ ]:
ads[['users_power', 'mean_users_power', 'mean_cpm_per_users']] = ads['user_ids'].apply(
    lambda user_ids: process_ad_users(user_ids, user_cpms)
).apply(pd.Series)

Здесь примерно то же самое, только обходим по площадкам

In [ ]:
def process_ad_publishers(publisher_ids, publisher_users):
    publishers = publisher_ids.split(',')
    counts = []
    for publisher in publishers:
        if publisher in publisher_users:
            counts.append(len(publisher_users[publisher]))

    return sum(counts), np.mean(counts)

In [ ]:
ads[['active_users_in_publishers', 'mean_active_users_in_publishers']] = ads['publishers'].apply(
    lambda publisher_ids: process_ad_publishers(publisher_ids, publisher_users)
).apply(pd.Series)

## Завершение обработки данных

Далее будем приводить всё к виду, в котором будет удобно скармливать данные модели

In [ ]:
features = ads.copy()

Тут удаляем совсем неподходящие (по типу) колонки

In [ ]:
features = features.drop(columns=['user_ids', 'publishers'])

Тут удаляем уже те, которые нам не нравятся (по влиянию на модель)

In [ ]:
features = features.drop(columns=['hour_start', 'hour_end'])

Вот что у нас получилось по составу колонок

In [ ]:
features.columns

Посмотрим на матрицу корреляций

In [ ]:
corr = features.corr()
corr.style.background_gradient(cmap='coolwarm')

Видно, что некоторые признаки сильно коррелируют, но я не стал их убирать пока, потому что они всё равно полезны для метрики

In [ ]:
target = validate_answers_df.copy()

Делим на train и test

In [ ]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.33, random_state=42)

## Работа с моделью

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score 

def print_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2: {r2:.4f}")

In [ ]:
import optuna
from sklearn.metrics import root_mean_squared_error

optuna.logging.set_verbosity(optuna.logging.WARNING)

Используем CatBoostRegressor, потому что модель дает неплохой результат сходу (лучше чем у xgboost и GradientBoostingRegressor)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
def optimize_catboost(X_train, X_test, y_train, y_test, n_trials=50):
    def objective(trial):
        params = {
            "iterations": trial.suggest_int("iterations", 100, 1000),
            "depth": trial.suggest_int("depth", 3, 10),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
            "border_count": trial.suggest_int("border_count", 32, 255),
            "random_strength": trial.suggest_float("random_strength", 1e-3, 10.0, log=True),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
            "loss_function": "RMSE",
            "eval_metric": "RMSE",
            "random_seed": 42,
            "silent": True
        }

        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        rmse = root_mean_squared_error(y_test, y_pred)
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_params, study.best_value

In [ ]:
X_train1, X_test1, y_train1, y_test1 = features_train, features_test, target_train['at_least_one'], target_test['at_least_one']

#params1 = optimize_catboost(X_train1, X_test1, y_train1, y_test1, 200)[0]

model1 = CatBoostRegressor(silent=True)
model1.fit(X_train1, y_train1)

y_true1 = np.array(y_test1)
y_pred1 = model1.predict(X_test1)

print_metrics(y_true1, y_pred1)

Посмотрим на значимость фичей  
Для этого будем использовать библиотеку SHAP

In [ ]:
import shap

Инициализируем библиотеку

In [ ]:
explainer = shap.TreeExplainer(model1)
shap_values = explainer(X_train1)

Выведем фичи и их значимость

In [ ]:
shap.plots.bar(shap_values)

Также выведем heatmap фич, чтобы увидеть как фичи описывают данные в совокупности (порядок семплов определен библиотекой на основе объясняемой схожести, т.е. библиотка применяет иерархическую кластеризацию для того, чтобы наиболее "объясняемые" с помощью наших фич были рядом друг с другом)

In [ ]:
shap.plots.heatmap(shap_values);

Можно убрать малозначимые и добавить каких-нибудь еще интересных

Я пробовал перемножать и делить базовые фичи (cpm, duration, sizes), но там ничего супер полезного не было

Неплохо  
У меня получалось лучше, но там было в 2 раза больше малополезных фичей, а результат не сильно отличался  

График распределения значений

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

result1 = pd.DataFrame()
result1['y_test'] = y_test1
result1['y_pred'] = y_pred1

sns.regplot(data=result1, x='y_test', y='y_pred', scatter_kws={"s": 20, "alpha": 0.5})

x = np.linspace(0, result1.max(), 100)
plt.plot(x, x, color="red");

Видно, что есть некоторая ошибка (расхождение между линиями), но в целом получается неплохо

Но это пока только один таргет, а нам надо еще 2

Во время анализа данных была выявлена сильная корреляция между таргетами  
Воспользуемся этим

Из всех опробованных моделей (GradientBoostingRegressor, CatBoostRegressor, LinearRegression) без оптимизации гиперпараметров лучше с задачей справляется GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
def optimize_gradient_boosting(X_train, X_test, y_train, y_test, n_trials=50):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        }

        model = GradientBoostingRegressor(**params, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        rmse = root_mean_squared_error(y_test, y_pred)
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_params, study.best_value

Попробуем обучить (пока независимо) модели для регрессии двух оставшихся таргетов

In [ ]:
X_train2, X_test2, y_train2, y_test2 = target_train[['at_least_one']], target_test[['at_least_one']], target_train['at_least_two'], target_test['at_least_two']

#params2 = optimize_gradient_boosting(X_train2, X_test2, y_train2, y_test2, 200)[0]

model2 = GradientBoostingRegressor()
model2.fit(X_train2, y_train2)

y_pred2 = model2.predict(X_test2)

print_metrics(y_test2, y_pred2)

In [ ]:
X_train3, X_test3, y_train3, y_test3 = target_train[['at_least_two']], target_test[['at_least_two']], target_train['at_least_three'], target_test['at_least_three']

#params3 = optimize_gradient_boosting(X_train3, X_test3, y_train3, y_test3, 200)[0]

model3 = GradientBoostingRegressor()
model3.fit(X_train3, y_train3)

y_pred3 = model3.predict(X_test3)

print_metrics(y_test3, y_pred3)

В целом получаются неплохие метрики

Теперь попробуем объединить модели в одну и посмотрим результат сразу на предложенной метрике

In [ ]:
from metrics import get_smoothed_mean_log_accuracy_ratio

В baseline (все нули) метрика равна

In [ ]:
baseline = pd.read_csv(os.path.join(data_dir, 'baseline.tsv'), sep='\t')
baseline_train, baseline_test = train_test_split(baseline, test_size=0.33, random_state=42)
get_smoothed_mean_log_accuracy_ratio(target_test, baseline_test)

Нам надо сделать лучше)

У модели есть параметр, который отвечает за то будет ли каждая "следующая" суб-модель обучаться на результатах прошлой модели или будем обучать их независимо

In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import GradientBoostingRegressor

class VKAdsRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, train_independently: bool = False):
        self.train_independently = train_independently
        self.model1 = CatBoostRegressor(silent=True)
        self.model2 = GradientBoostingRegressor()
        self.model3 = GradientBoostingRegressor()

    def fit(self, X, y):
        self.model1.fit(X, y['at_least_one'])
        
        if self.train_independently:
            self.model2.fit(y[['at_least_one']].to_numpy(), y['at_least_two'])
            self.model3.fit(y[['at_least_two']].to_numpy(), y['at_least_three'])
        else:
            output1 = np.clip(self.model1.predict(X), a_min=0, a_max=1).reshape(-1, 1)
            self.model2.fit(output1, y['at_least_two'])

            output2 = self.model2.predict(output1).reshape(-1, 1)
            self.model3.fit(output2, y['at_least_three'])

        return self

    def predict(self, X):
        output1 = np.clip(self.model1.predict(X), a_min=0, a_max=1)
        output2 = self.model2.predict(output1.reshape(-1, 1))
        output3 = self.model3.predict(output2.reshape(-1, 1))

        output = pd.DataFrame()
        output['at_least_one'] = output1
        output['at_least_two'] = output2
        output['at_least_three'] = output3
        output = output.set_index(X.index)
        
        return output

Для начала попробуем обучать независимо

In [ ]:
model = VKAdsRegressor(train_independently=True)
model.fit(features_train, target_train)

target_pred = model.predict(features_test)

get_smoothed_mean_log_accuracy_ratio(target_test, target_pred)

А теперь попробуем во время обучения постараться учесть ошибки модели и каждую следующую суб-модель будем обучать, используя данные предыдущей суб-модели

In [ ]:
model = VKAdsRegressor(train_independently=False)
model.fit(features_train, target_train)

target_pred = model.predict(features_test)

get_smoothed_mean_log_accuracy_ratio(target_test, target_pred)

Немного странно, но получилось так что независимое обучение суб-моделей дало лучший результат, но возможно при оптимизации гиперпараметров модели всё изменится

В итоге получили результат ~50 по предложенной метрике, что однозначно лучше чем baseline)